In [1]:
! pip install torch torchvision transformers datasets pillow

   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   --- ------------------------------------ 2.4/25.3 MB 12.2 MB/s eta 0:00:02
   -------- ------------------------------- 5.2/25.3 MB 12.8 MB/s eta 0:00:02
   ------------- -------------------------- 8.7/25.3 MB 13.8 MB/s eta 0:00:02
   ------------------- -------------------- 12.3/25.3 MB 14.9 MB/s eta 0:00:01
   --------------------------- ------------ 17.3/25.3 MB 16.5 MB/s eta 0:00:01
   ---------------------------------- ----- 21.5/25.3 MB 17.0 MB/s eta 0:00:01
   ---------------------------------------  25.2/25.3 MB 17.5 MB/s eta 0:00:01
   ---------------------------------------- 25.3/25.3 MB 16.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import BlipProcessor, BlipForConditionalGeneration, TrainingArguments, Trainer
from PIL import Image

# ✅ Set Device (Use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ✅ Load Dataset
dataset = load_dataset("kkcosmos/instagram-images-with-captions", split="train")

# ✅ Load BLIP Model & Processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
def preprocess_data(example):
    try:
        image = example["image"].convert("RGB")  # Ensure RGB format
        encoding = processor(images=image, text=example["caption"], padding="max_length", truncation=True, return_tensors="pt")

        return {
            "pixel_values": encoding["pixel_values"].squeeze(),
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze()
        }
    except Exception as e:
        print(f"Skipping image due to error: {e}")
        return None  # Skip corrupted images

# ✅ Apply Preprocessing
dataset = dataset.map(preprocess_data, remove_columns=["image", "caption"], batched=False)
dataset = dataset.filter(lambda x: x is not None) 